In [1]:
import numpy as np
import pandas as pd

## 五、Pandas数据合并

- pd.concat
- pd.append
- pd.merge 

为了方便，我们首先定义一个生成DataFrame的函数：
### 1. 使用pd.concat()级联

Pandas使用pd.concat函数，与NumPy中的concatenate函数类似，只是多了一些参数：

#### 1) 简单级联

#### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

### 2. 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

### 3.使用merge()合并

- 类似MySQL中表和表之间的合并 
- merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并
- 使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。
- 每一列元素的顺序不要求一致

#### 1) 一对一合并

#### 2) 多对一合并

#### 3) 多对多合并

#### 4) key的规范化

- 使用on=显式指定哪一列为key,当有多个key相同时使用
- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用
- 当左边的列和右边的index相同的时候,使用right_index=True

#### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）
- 外合并 how='outer'：补NaN
- 左合并、右合并：how='left'，how='right'

#### 6) 添加后缀

- 当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名
- 可以使用suffixes=自己指定后缀

#### merge合并总结:

- 合并有三种现象: 一对一, 多对一, 多对多.
- 合并默认会找相同的列名进行合并, 如果有多个列名相同,用on来指定.
- 如果没有列名相同,但是数据又相同,可以通过left_on, right_on来分别指定要合并的列.
- 如果想和index合并, 使用left_index, right_index来指定.
- 如果多个列相同,合并之后可以通过suffixes来区分.
- 还可以通过how来控制合并的结果, 默认是内合并, 还有外合并outer, 左合并left, 右合并right.

- pd.concat
- pd.append
- pd.merge 

为了方便，我们首先定义一个生成DataFrame的函数：

In [2]:
def make_df(indexs, columns):    
    data = [[str(j)+str(i) for j in columns] for i in indexs]
    df = pd.DataFrame(data=data, index=indexs, columns=columns)
    return df

In [3]:
make_df([1, 2, 3, 4], list('ABCD'))

,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


### 1. 使用pd.concat()级联

Pandas使用pd.concat函数，与NumPy中的concatenate函数类似，只是多了一些参数：

#### 1) 简单级联
参数：

- ignore_index：忽略索引，重置索引

#### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

In [5]:
df1 = make_df([1,2],['A','B'])
df2 = make_df([3,4],['A','B'])
display(df1,df2)

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


In [8]:
# 默认上下（垂直）合并
pd.concat([df1,df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [10]:
# 左右合并，水平合并，索引不匹配则新增列和行，索引位置不存在值默认为NaN
pd.concat([df1,df2],axis=1)

,A,B,A,B
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,NaN,NaN,A3,B3
4,NaN,NaN,A4,B4


- ignore_index：忽略索引，重置索引

In [17]:
display(pd.concat([df1,df2], ignore_index=True))
display(pd.concat([df1,df2], axis=1, ignore_index=True))

,A,B
0,A1,B1
1,A2,B2
2,A3,B3
3,A4,B4


,0,1,2,3
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,NaN,NaN,A3,B3
4,NaN,NaN,A4,B4


- keys：使用多层索引

In [15]:
display(pd.concat([df1,df2],keys=['x','y']))
display(pd.concat([df1,df2],axis=1,keys=['x','y']))

A   B
x 1  A1  B1
  2  A2  B2
y 3  A3  B3
  4  A4  B4

x         y     
     A    B    A    B
1   A1   B1  NaN  NaN
2   A2   B2  NaN  NaN
3  NaN  NaN   A3   B3
4  NaN  NaN   A4   B4